In [2]:
from hathor import Hathor
import sys
import os
hathor_root = os.path.dirname(os.getcwd())
sys.path.insert(0, hathor_root)
from inference_auth_token import get_access_token
access_token = get_access_token()
config_list = [{
    "model": "openai/gpt-oss-120b",  # or whatever model name your endpoint expects
    "api_key": access_token,
    "base_url": "https://inference-api.alcf.anl.gov/resource_server/sophia/vllm/v1",
    "api_type": "openai",
    "max_tokens": 4000, 
}]

prompt = """
            The data provided to you comes from the MEGATRON simulations, but you have only been given the gas and star file for ONE small halo at ONE time step.
        """

hathor = Hathor(prompt=prompt, config_list=config_list)
hathor.run()

UserProxy (to chat_manager):


                        Please generate a high quality galaxy formation hypothesis that can be evaluated using RAMSES simulation data.                         Please also generate a high quality plot idea that will use RAMSES data to evaluate that hypothesis.
                    

--------------------------------------------------------------------------------


InternalServerError: Error code: 500 - Error: Could not recover future result: TaskExecutionFailed('Traceback (most recent call last):\n  File "/home/openinference_svc/register_functions/vllm_register_function_with_streaming_for_openai.py", line 422, in vllm_inference_function\n    return handle_non_streaming_request(url, headers, payload, start_time, is_health_check)\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home/openinference_svc/register_functions/vllm_register_function_with_streaming_for_openai.py", line 173, in handle_non_streaming_request\n    raise Exception(error_msg)\nException: API request failed with status code: 400\nResponse text: {"error":{"message":"max_tokens must be at least 1, got -18568.","type":"BadRequestError","param":null,"code":400}}\nResponse headers: {\'date\': \'Wed, 03 Dec 2025 21:07:43 GMT\', \'server\': \'uvicorn\', \'content-length\': \'115\', \'content-type\': \'application/json\'}\n\nDuring ha

In [ ]:
import arxiv
import numpy as np

import requests
from io import BytesIO
import fitz

In [ ]:
client = arxiv.Client()
search =arxiv.Search(query="circumgalactic medium",  
                        max_results=20,
                        sort_by=arxiv.SortCriterion.SubmittedDate,
                        sort_order=arxiv.SortOrder.Descending 
                    )
results = list(client.results(search))

chosen_papers = np.random.choice(results, size=5, replace=False)

search =arxiv.Search(query="circumgalactic medium",  
                        max_results=20,
                        sort_by=arxiv.SortCriterion.Relevance,
                        sort_order=arxiv.SortOrder.Descending 
                    )
results = list(client.results(search))
relevant_papers = np.random.choice(results, size=5, replace=False)

chosen_papers = np.concatenate([chosen_papers,relevant_papers])

for paper in chosen_papers:
    # Download and extract full text
    try:
        response = requests.get(paper.pdf_url)
        pdf_file = BytesIO(response.content)
        
        doc = fitz.open(stream=pdf_file, filetype="pdf")
        full_text = ""
        for page in doc:
            full_text += page.get_text()
        doc.close()
    except Exception as e:
        full_text = f"[Error extracting text: {e}]"
    
    # Create paper string with correct attributes
    paper_string = f"""
                    Title: {paper.title}
                    Abstract: {paper.summary}
                    Full Text: {full_text}
                    """
    print(paper_string)
 



                    Title: Phononic Casimir Effect in Planar Materials
                    Abstract: The Phononic Casimir effect between planar objects is investigated by deriving a formalism from the quantum partition function of the system following multiscattering approach. This fluctuation-induced coupling is mediated by phonons modeled as an effective elastic medium. We find that excitations with three types of polarizations arise from the resolved boundary conditions, however the coupling is dominated by only one of these degrees of freedom due to exponential suppression effects in the other two. The obtained scaling laws and dependence on materials properties and temperature suggest effective pathways of interaction control. Scenarios of materials combinations are envisioned where the Phononic Casimir effect is of similar order as the standard Casimir interaction mediated by electromagnetic fluctuations.
                    Full Text: [Error extracting text: name 'requests' is 